In [1]:
# example from https://towardsdatascience.com/replicate-a-logistic-regression-model-as-an-artificial-neural-network-in-keras-cd6f49cf4b2c

In [2]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [3]:
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data,
                  columns=cancer.feature_names)
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
X = (df-df.mean())/df.std()
# X = 2*(df-df.min())/(df.max()-df.min()) - 1
# X = df
y = pd.Series(cancer.target)

In [5]:
# change 0 to -1
# y = y.apply(lambda x: -1 if x==0 else 1)

In [6]:
# Make train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    shuffle=True, random_state=2)

In [7]:
# Initialize a logistic regression model
log_reg_model = LogisticRegression(max_iter=2500,
                                   random_state=42)

# Train (fit) the model
log_reg_model.fit(X_train, y_train)

# Make predictions
y_pred = log_reg_model.predict(X_test) # Predictions
y_true = y_test # True values

# Model evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

print("Accuracy:", np.round(accuracy_score(y_true, y_pred), 2))
precision, recall, fscore, _ = precision_recall_fscore_support(y_true, y_pred,
                                                               average='binary')
print("Precision:", np.round(precision, 2))
print("Recall:", np.round(recall, 2))
print("F-Score:", np.round(fscore, 2))

Accuracy: 0.97
Precision: 0.99
Recall: 0.97
F-Score: 0.98


In [8]:
model = Sequential()

model.add(InputLayer(input_shape=(30, )))
# No hidden layers
model.add(Dense(1, activation='tanh'))

optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.05)
model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train,
          epochs=10, batch_size=32,
          validation_split=0.2,
          shuffle=False)

Epoch 1/10


2023-10-12 14:44:16.741589: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


12/12 [==============================] - 0s 12ms/step - loss: 2.0170 - accuracy: 0.7747 - val_loss: 0.9741 - val_accuracy: 0.8901
Epoch 2/10
12/12 [==============================] - 0s 3ms/step - loss: 1.0516 - accuracy: 0.8929 - val_loss: 0.7816 - val_accuracy: 0.9011
Epoch 3/10
12/12 [==============================] - 0s 2ms/step - loss: 0.8193 - accuracy: 0.9093 - val_loss: 0.7534 - val_accuracy: 0.9121
Epoch 4/10
12/12 [==============================] - 0s 3ms/step - loss: 0.6689 - accuracy: 0.9341 - val_loss: 0.7462 - val_accuracy: 0.9121
Epoch 5/10
12/12 [==============================] - 0s 3ms/step - loss: 0.6168 - accuracy: 0.9423 - val_loss: 0.7404 - val_accuracy: 0.9231
Epoch 6/10
12/12 [==============================] - 0s 2ms/step - loss: 0.3145 - accuracy: 0.9560 - val_loss: 0.2055 - val_accuracy: 0.9670
Epoch 7/10
12/12 [==============================] - 0s 2ms/step - loss: 0.2109 - accuracy: 0.9505 - val_loss: 0.2063 - val_accuracy: 0.9670
Epoch 8/10
12/12 [============

In [10]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

4/4 [==============================] - 0s 1ms/step - loss: 0.4077 - accuracy: 0.9474
Test loss: 0.4076523780822754
Test accuracy: 0.9473684430122375


In [11]:
def zanh(x):
    return 0.006769816 + 0.554670504 * x - 0.009411195 * x**2 - 0.014187547 * x**3

zanh_model = Sequential()

zanh_model.add(InputLayer(input_shape=(30, )))
# No hidden layers
zanh_model.add(Dense(1, activation=zanh))

optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.05)
zanh_model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [12]:
zanh_model.fit(X_train, y_train,
          epochs=10, batch_size=32,
          validation_split=0.2,
          shuffle=False)

Epoch 1/10
12/12 [==============================] - 0s 12ms/step - loss: 2.7550 - accuracy: 0.6978 - val_loss: 1.5429 - val_accuracy: 0.7912
Epoch 2/10
12/12 [==============================] - 0s 3ms/step - loss: 1.5446 - accuracy: 0.8297 - val_loss: 0.5988 - val_accuracy: 0.9341
Epoch 3/10
12/12 [==============================] - 0s 3ms/step - loss: 0.6857 - accuracy: 0.8791 - val_loss: 0.1382 - val_accuracy: 0.9670
Epoch 4/10
12/12 [==============================] - 0s 3ms/step - loss: 0.2950 - accuracy: 0.9258 - val_loss: 0.1452 - val_accuracy: 0.9451
Epoch 5/10
12/12 [==============================] - 0s 3ms/step - loss: 0.2493 - accuracy: 0.9478 - val_loss: 0.1188 - val_accuracy: 0.9670
Epoch 6/10
12/12 [==============================] - 0s 3ms/step - loss: 0.1930 - accuracy: 0.9588 - val_loss: 0.1023 - val_accuracy: 0.9670
Epoch 7/10
12/12 [==============================] - 0s 3ms/step - loss: 0.1749 - accuracy: 0.9643 - val_loss: 0.0879 - val_accuracy: 0.9670
Epoch 8/10
12/12 [=

In [13]:
test_loss, test_acc = zanh_model.evaluate(X_test, y_test)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

4/4 [==============================] - 0s 1ms/step - loss: 0.2434 - accuracy: 0.9649
Test loss: 0.24344508349895477
Test accuracy: 0.9649122953414917
